# Potencial tipo embudo 3D

El potencial está todavía por reajustar. Esto se hizo con apoyo del notebook Gaussian_kernel.

In [1]:
import numpy as np
import sympy as sy
import matplotlib.pyplot as plt
import simtk.openmm as mm
import simtk.unit as unit
import simtk.openmm.app as app
from tqdm import tqdm

In [2]:
plt.style.use('ggplot')

# Definición del sistema

In [3]:
system = mm.System()
mass   = 39.948 * unit.amu # masa del átomo Ar
system.addParticle(mass)

0

# Potencial tipo embudo 3D

In [4]:
def elliptical_gaussian_kernel_3D(x,y,z,xo,yo,zo,A=-1.0,cxx=0.0,cxy=0.0,cxz=0.0,
                                  cyy=0.0,cyz=0.0,czz=0.0):
    inc_x = x-xo
    inc_y = y-yo
    inc_z = z-zo
    B = cxx*inc_x*inc_x + cyy*inc_y*inc_y + czz*inc_z*inc_z + \
        2*cxy*inc_x*inc_y + 2*cxz*inc_x*inc_z + 2*cyz*inc_y*inc_z
    return A*np.exp(-B)

def check_positive_3D(embudo):
    
    pot_i=0
    for pot in embudo:
        xo, yo, zo, A, cxx, cxy, cxz, cyy, cyz, czz = pot
        M = np.array([[cxx, cxy, cxz], [cxy, cyy, cyz], [cxz, cyz, czz]])
        is_positive = np.all(np.linalg.eigvals(M+M.transpose()) > 0)
        if not is_positive:
            print('La gaussiana',pot_i, 'no es definida positiva')
        pot_i+=1

        
def effective_ks_3D(embudo):
    
    effective_ks = []
    pot_i=0
    for pot in embudo:

        xo, yo, zo, A, cxx, cxy, cxz, cyy, cyz, czz = pot
        x = sy.symbols('x')
        y = sy.symbols('y')
        z = sy.symbols('z')
        f = A*sy.exp(-(cxx*(x-xo)**2 + 2*cxy*(x-xo)*(y-yo) + 2*cxz*(x-xo)*(z-zo) + cyy*(y-yo)*(y-yo) + 2*cyz*(y-yo)*(z-zo) + czz*(z-zo)**2 ))
        gradient = sy.derive_by_array(f,(x,y,z))
        hessian = sy.derive_by_array(gradient,(x,y,z))
        M = hessian.subs(x,xo).subs(y,yo).subs(z,zo)
        M = np.array(M.tolist(),dtype=float)
        eigenvals, eigenvecs = np.linalg.eig(np.array(M))
        #print('Gaussiana',pot_i)
        for ii in range(eigenvals.shape[0]):
            eff_k = eigenvals[ii]
            if eff_k > 0.0:
                effective_ks.append(eff_k)
            #print('\t effective k',eff_d,'en modo',eigenvecs[:,ii])
        pot_i+=1
        #print('')
    return np.max(effective_ks)



In [5]:
def plot_embudo_mayavi_3D(embudo):
    
    xlim = [-10,10]
    ylim = [-10,10]
    zlim = [-10,10]
    xbins = 200
    ybins = 200
    zbins = 200

    x = np.linspace(xlim[0], xlim[1], xbins)
    y = np.linspace(ylim[0], ylim[1], ybins)
    z = np.linspace(zlim[0], zlim[1], zbins)

    X,Y,Z = np.meshgrid(x, y, z)
    scalars = 0.0
    for pot in embudo:
        xo, yo, zo, A, cxx, cxy, cxz, cyy, cyz, czz = pot
        scalars += elliptical_gaussian_kernel_3D(X,Y,Z,xo,yo,zo,A,cxx,cxy,cxz,cyy,cyz,czz)


    return mlab.contour3d(scalars, contours=14, opacity = 0.2)

In [6]:
#pot_i = [  xo,   yo,  zo,    A,  cxx,  cxy, cxz, cyy, cyz, czz]
pot_0 =  [ 0.0,  0.0, 0.0, -2.0, 0.05,  0.0, 0.0, 0.05, 0.0, 0.05]
pot_1 =  [ 0.0,  0.0, 0.0, -0.6,  2.0,  0.0, 0.0,  2.0, 0.0, 2.0]
pot_2 =  [ 3.0,  3.0, 3.0, -0.8,  1.0,  0.5, 0.2,  1.0, 0.1, 1.0]
pot_3 =  [ 0.0,  5.0, 0.0, -0.6,  1.0,  0.0, 0.0,  1.0, 0.2, 1.2]
pot_4 =  [-2.5, -2.0,-2.0, -0.5,  1.0,  0.0, 0.4,  1.0, 0.0, 1.2]
pot_5 =  [ 2.2, -2.0, 2.0, -0.8,  2.0,  0.0, 0.2,  2.0, 0.4, 0.8]
pot_6 =  [ 3.0,  6.0, 3.0, -0.8,  2.0,  0.8, 0.0,  2.0, 0.2, 2.0]
pot_7 =  [-4.5,  1.0, 2.0, -0.8,  2.0,  0.0, 0.1,  2.0, 0.0, 0.7]
pot_8 =  [-3.0,  4.0, 1.0, -0.6,  1.0,  0.4, 0.0,  0.7, 0.0, 1.0]
pot_9 =  [ 0.0, -4.0, 0.0, -0.8,  2.0,  0.2, 0.2,  2.0, 0.3, 2.0]
pot_10 = [-2.0,  5.8,-3.0, -0.8,  1.0,  0.1, 0.0,  1.0, 0.0, 1.2]
pot_11 = [-1.8,  0.8, 3.0, -0.5,  0.9,  0.0, 0.2,  2.0, 0.2, 1.5]
pot_12 = [ 5.0, -0.3, 5.0, -0.8,  1.0,  0.4, 0.3,  1.0, 0.0, 1.2]
pot_13 = [ 4.0, -4.0, 4.0, -0.8,  1.0,  0.0, 0.0,  1.0, 0.1, 0.7]
pot_14 = [ 5.0, -2.5,-4.0, -0.6,  2.0,  0.0, 0.3,  1.0, 0.0, 0.8]
pot_15 = [-3.0, -4.0,-4.0, -1.0,  0.8,  0.3, 0.0,  1.0, 0.3, 1.0]
pot_16 = [-4.0,  3.0, 4.0, -0.7,  3.5,  0.0, 0.4,  3.5, 0.0, 1.2]
pot_17 = [-5.0, -2.0, 6.0, -1.2,  1.5,  0.0, 0.0,  1.5, 0.2, 0.8]
pot_18 = [-6.0, -1.8,-6.0, -0.8,  1.0,  0.2, 0.1,  0.8, 0.0, 1.0]
pot_19 = [ 2.0, -5.0, 5.0, -1.0,  1.5,  0.0, 0.1,  1.5, 0.1, 1.5]
pot_20 = [-6.0,  0.0, 2.0, -1.0,  1.5,  0.1, 0.0,  1.5, 0.0, 0.9]
pot_21 = [-6.0,  6.0, 1.0, -1.2,  1.0,  0.5, 0.2,  0.9, 0.2, 0.9]
pot_22 = [ 6.2,  2.0, 3.0, -1.2,  1.0,  0.0, 0.0,  1.0, 0.2, 1.0]
pot_23 = [ 2.5,  0.5, 0.0, -1.0,  1.5,  0.0, 0.1,  1.5, 0.2, 1.5]
pot_24 = [-4.0, -1.0, 3.0, -1.0,  1.2,  0.1, 0.0,  1.5, 0.2, 1.2]
pot_25 = [-1.5, -2.0, 3.0, -0.9,  0.7,  0.5, 0.1,  0.9, 0.0, 0.9]
pot_26 = [-4.2,  2.0,-3.0, -1.2,  1.2,  0.0, 0.3,  0.8, 0.1, 0.8]
pot_27 = [ 4.0,  4.0, 3.0, -0.8,  0.8,  0.5, 0.0,  0.9, 0.0, 1.0]
pot_28 = [ 4.2, -2.0,-3.0, -0.8,  1.0,  0.0, 0.4,  1.0, 0.2, 1.0]
pot_29 = [-6.0, -1.8,-6.0, -0.8,  1.0,  0.2, 0.1,  0.8, 0.0, 1.0]
pot_30 = [ 2.0, -5.0, 5.0, -1.0,  1.5,  0.0, 0.1,  1.5, 0.1, 1.5]
pot_31 = [-6.0,  5.0, 2.0, -1.0,  0.8,  0.1, 0.0,  1.5, 0.0, 0.9]
pot_32 = [ 6.0,  6.0, 1.0, -1.2,  0.8,  0.5, 0.2,  0.9, 0.2, 0.9]
pot_33 = [ 0.2,  5.0, 5.0, -1.2,  1.0,  0.0, 0.0,  0.5, 0.6, 1.0]
pot_34 = [-4.0,  2.0, 5.0, -1.2,  1.5,  0.0, 0.2,  1.0, 0.3, 1.5]
pot_35 = [-3.0, -1.8,-2.0,  0.4,  1.0,  0.2, 0.1,  0.8, 0.0, 1.0]
pot_36 = [ 2.0, -3.0, 4.0,  0.6,  1.5,  0.0, 0.1,  1.5, 0.1, 1.5]
pot_37 = [ 3.0,  0.0, 1.0,  0.4,  0.8,  0.1, 0.0,  1.5, 0.0, 0.9]
pot_38 = [ 2.0, -4.0,-1.0,  0.6,  0.8,  0.5, 0.2,  0.9, 0.2, 0.9]
pot_39 = [ 0.2,  3.0, 3.0,  0.4,  1.0,  0.0, 0.0,  0.5, 0.6, 1.0]
pot_40 = [-1.0,  2.0, 3.0,  0.6,  1.5,  0.0, 0.2,  1.0, 0.3, 1.5]


embudo = [pot_0, pot_1, pot_2, pot_3, pot_4, pot_5, pot_6, pot_7, pot_8, pot_9, 
         pot_10, pot_11, pot_12, pot_13, pot_14, pot_15, pot_16, pot_17, pot_18,
         pot_19, pot_20, pot_21, pot_22, pot_23, pot_24, pot_25, pot_26, pot_27,
         pot_28, pot_29, pot_30, pot_31, pot_32, pot_33, pot_34, pot_35, pot_36,
         pot_37, pot_38, pot_39, pot_40]

check_positive_3D(embudo)
effective_ks_3D(embudo)

4.994611392664028

Calculamos el periodo de oscilación natural en lo mínimos de los kernel gaussianos usados para construir el potencial embudo. Tomamos la máxima k efectiva para tener referencia temporal para la elección del paso de integración.

In [7]:
effective_k = effective_ks_3D(embudo)
T = 2*np.pi*np.sqrt(mass/(effective_k * unit.kilocalories_per_mole/unit.angstrom**2))
print('Periodo T de oscilación',T)

Periodo T de oscilación 17.769554396705637


### Añadiendo el potencial al sistema

In [9]:
### Potencial tipo embudo en X, Y y Z

gaussian_kernel = 'A*exp(-(cxx*(x-xo)^2+cyy*(y-yo)^2+czz*(z-zo)^2+ \
                   2.0*cxy*(x-xo)*(y-yo)+2.0*cxz*(x-xo)*(z-zo)+2.0*cyz*(y-yo)*(z-zo)))'

for pot in embudo:
    xo, yo, zo, A, cxx, cxy, cxz, cyy, cyz, czz = pot
    force = mm.CustomExternalForce(gaussian_kernel)
    force.addGlobalParameter('A', A * unit.kilocalories_per_mole)
    force.addGlobalParameter('cxx', cxx * 1.0/unit.angstrom**2)
    force.addGlobalParameter('cxy', cxy * 1.0/unit.angstrom**2)
    force.addGlobalParameter('cxz', cxz * 1.0/unit.angstrom**2)
    force.addGlobalParameter('cyy', cyy * 1.0/unit.angstrom**2)
    force.addGlobalParameter('cyz', cyz * 1.0/unit.angstrom**2)
    force.addGlobalParameter('czz', czz * 1.0/unit.angstrom**2)
    force.addGlobalParameter('xo', xo * unit.angstrom)
    force.addGlobalParameter('yo', yo * unit.angstrom)
    force.addGlobalParameter('zo', zo * unit.angstrom)
    force.addParticle(0, [])
    system.addForce(force)

# Estado termodinámico

In [ ]:
# Formalismo NVT
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
temperature = 300*unit.kelvin
pressure    = None

# Parámetros de la simulación

In [ ]:
step_size       = 0.01*unit.picoseconds
num_steps       = 4000000
saving_period   = 100
num_steps_saved = int(num_steps/saving_period)

# Integrador

In [ ]:
friction   = 1.0/unit.picosecond
integrator = mm.LangevinIntegrator(temperature, friction, step_size)

# Plataforma de simulación

In [ ]:
platform_name = 'CPU'  #platform:     A platform (CPU, OpenCL, CUDA, or reference); default is platform=OpenCL"
# for ii in range(mm.Platform.getNumPlatforms()):
#     print(mm.Platform.getPlatform(ii).getName())
platform = mm.Platform.getPlatformByName(platform_name)

# Reporteros y arrays de salida

In [ ]:
times      = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.picoseconds)
positions  = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms)
velocities = unit.Quantity(np.zeros([num_steps_saved,3], np.float32), unit.angstroms/unit.picosecond)
potential_energies   = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)
kinetic_energies     = unit.Quantity(np.zeros([num_steps_saved], np.float32), unit.kilocalories_per_mole)

# Condiciones iniciales

In [ ]:
initial_positions  = [[0.0, 0.0, 0.0]] * unit.angstroms
#initial_velocities = None # Las velocidades serán adjudicadas aleatoriamente según la distribución Maxwell-Boltzmann del estado termodinámico

context = mm.Context(system, integrator, platform)
context.setPositions(initial_positions)
context.setVelocitiesToTemperature(temperature)

In [ ]:
state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
times[0] = state.getTime()
positions[0] = state.getPositions()[0]
velocities[0] = state.getVelocities()[0]
kinetic_energies[0]=state.getKineticEnergy()
potential_energies[0]=state.getPotentialEnergy()

# Corriendo la simulación

In [ ]:
for ii in tqdm(range(num_steps_saved)):
    context.getIntegrator().step(saving_period)
    state = context.getState(getEnergy=True, getPositions=True, getVelocities=True)
    times[ii] = state.getTime()
    positions[ii] = state.getPositions()[0]
    velocities[ii] = state.getVelocities()[0]
    kinetic_energies[ii]=state.getKineticEnergy()
    potential_energies[ii]=state.getPotentialEnergy()

# Análisis de resultados

In [ ]:
plt.rcParams['figure.figsize'] = 18, 4
for ii, ylabel in zip(range(3),['X','Y','Z']):
    plt.plot(times,positions[:,ii])
    plt.ylabel(ylabel)
    plt.xlabel('time')
    plt.show()

In [ ]:
hist, bin_edges = np.histogram(positions[:,0],bins=100, density=True)
plt.plot((bin_edges[1:]+bin_edges[:-1])/2.0,hist)
plt.show()

In [ ]:
plt.plot((bin_edges[1:]+bin_edges[:-1])/2.0,-np.log(hist))
plt.show()

In [ ]:
num_frames = positions.shape[0]
averages   = unit.Quantity(np.zeros(num_frames, np.float32), unit.angstroms)
std_devs   = unit.Quantity(np.zeros(num_frames, np.float32), unit.angstroms)
    
for ii in np.arange(num_frames):
    averages[ii] = np.mean(positions[:(ii+1),0])
    std_devs[ii]  = np.std(positions[:(ii+1),0])

plt.plot(times,averages)
plt.show()

plt.plot(times,std_devs)
plt.show()

In [ ]:
writing = False

if writing: 
    h5f = h5py.File('data_double_well_md.h5', 'w')
    h5f.create_dataset('times', data=times)
    h5f.create_dataset('positions', data=positions)
    h5f.create_dataset('velocities', data=velocities)
    h5f.create_dataset('kinetic_energies', data=kinetic_energies)
    h5f.create_dataset('potential_energies', data=potential_energies)
    h5f.close()

In [ ]:
reading =False

if reading:
    h5f = h5py.File('data_double_well_md.h5', 'r')
    #for name in h5f:
    #    print(name)
    times              = h5f['times'][:]
    positions          = h5f['positions'][:]
    velocities         = h5f['velocities'][:]
    kinetic_energies   = h5f['kinetic_energies'][:]
    potential_energies = h5f['potential_energies'][:]
    h5f.close()

In [ ]:
"""
T = sqrt(m/K)
timestep smaller than ~ T/10
standard deviation in each dimension sigma= (kT / K)^(1/2)
Expectation and standard deviation of the potential energy of a 3D harmonic oscillator is (3/2)kT
"""

### Todo:

- Los gaussian kernel son demasiado abiertos, la k efectiva es muy baja en comparación con el potencial del eje z